# Matrix testing

In [ ]:
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

DB_URL = "sqlite:///../../bleau_info-17-09-2025.db"

engine = create_engine(DB_URL, echo=False)

session = Session(engine)

In [2]:
import sys

sys.path.append("../../")

In [3]:
from scipy.sparse import load_npz

similarity_ascents = load_npz("../../similarity_ascent.npz")
similarity_style = load_npz("../../similarity_style.npz")
similarity_grade = load_npz("../../similarity_grade.npz")

In [4]:
import numpy as np
from sqlalchemy import select

from models.boulder import Boulder


def recommend_boulders(
    input_boulders,
    top_n=5,
    ascent_weight=0.5,
    style_weight=0.25,
    grade_weight=0.25,
):

    ascents = similarity_ascents[:, input_boulders].sum(axis=1).A1
    style = similarity_style[:, input_boulders].sum(axis=1).A1
    grade = similarity_grade[:, input_boulders].sum(axis=1).A1

    # Remove input boulders from the recommendation
    ascents[input_boulders] = 0
    style[input_boulders] = 0
    grade[input_boulders] = 0

    sim_scores = (
        ascent_weight * ascents + style_weight * style + grade_weight * grade
    )

    best_boulders = np.argsort(-sim_scores)[:top_n]

    return best_boulders.tolist()


ids = recommend_boulders(
    [12711], top_n=20, ascent_weight=0.5, style_weight=0.25, grade_weight=0.25
)

boulders = session.execute(
    select(Boulder.name, Boulder.id).filter(Boulder.id.in_(ids))
).all()

boulder_dict = {
    boulder_id: boulder_name for boulder_name, boulder_id in boulders
}
boulder_names = [(boulder_dict[boulder_id], boulder_id) for boulder_id in ids]
display(boulder_names)

[('Ça Tend à Gauche', 26610),
 ('Le Sacre du Printemps', 23907),
 ('Le Tri-Phasé', 34816),
 ('Freestyle', 34419),
 ('Maracuja', 18816),
 ('Freak of the Weak', 15554),
 ('Le Dolmen du Divin', 32911),
 ('Nouvelle Vague', 24491),
 ('La Maison de la Presse', 22541),
 ('Nazaré', 5408),
 ('Blozone', 22497),
 ('Für Alina (assis)', 3339),
 ('Hadès', 4634),
 ('Priapisme (assis)', 3338),
 ('Velociraptor', 27710),
 ('Goruliak (assis)', 4403),
 ('Le Parfum des Couleurs', 31679),
 ('Irréversible', 21768),
 ('Belle de Jour', 34129),
 ("Jeu t'es Fou (assis)", 20087)]